In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama 

from langchain.cache import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

llm = Ollama(model="llama3")


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14924\3452351312.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [9]:
import os
print(os.getcwd())

c:\Users\Usuario\Desktop\GitHub\LBW_project\lbw_project\Modeling


In [10]:
# List of PDF file paths
path = "C:\\Users\\Usuario\\Desktop\\GitHub\\LBW_project\\lbw_project\\Modeling\\pdf_files\\"
pdf_files = ["lbw_guidelines.pdf", "lbw_languagedevelopment.pdf"]

# Initialize an empty list to store documents
documents = []

# Load each PDF file
for pdf_file in pdf_files:
    loader = PyPDFLoader(path + pdf_file)
    documents.extend(loader.load())

In [12]:
documents

[Document(metadata={'source': 'C:\\Users\\Usuario\\Desktop\\GitHub\\LBW_project\\lbw_project\\Modeling\\pdf_files\\lbw_guidelines.pdf', 'page': 0}, page_content='Commentary\nLow birth weight: Case deﬁnition & guidelines for data collection,\nanalysis, and presentation of maternal immunization safety data\nClare L. Cutlanda,b,c,⇑, Eve M. Lackritzd, Tamala Mallett-Mooree, Azucena Bardajíf,\nRavichandran Chandrasekarang, Chandrakant Lahariyah, Muhammed Imran Nisari,\nMilagritos D. Tapiaj, Jayani Pathiranaa,b,c, Sonali Kochhark,m,1, Flor M. Muñozl,\nThe Brighton Collaboration Low Birth Weight Working Group2\na Medical Research Council: Respiratory and Meningeal Pathogens Research Unit, Johannesburg, South Africa\nb Department of Science and Technology National Research Foundation, Vaccine Preventable Diseases, South Africa\nc Faculty of Health Sciences, University of the Witwatersrand, Johannesburg, South Africa\nd Global Alliance to Prevent Prematurity and Stillbirth (GAPPS), Seattle Chil

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

chunks = text_splitter.split_documents(documents)

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="llama3",
)

db = Chroma.from_documents(
    documents = chunks,
    embedding = embedding,
    persist_directory = "db")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14924\3868745584.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(


In [ ]:
pergunta = "O que é baixo peso ao nascer?"

db.similarity_search_with_relevance_scores(pergunta, k = 3)

In [19]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(""" 
Answer the user query based on context. If you don't know the answer or the context don't have the answer, say you don't know. Always answer in pt-BR.
                          
## Context
{contexto}
                          
## USER QUERY
{pergunta}
""")


In [23]:
def rag(user_query: str) -> str:
    context = db.similarity_search_with_relevance_scores(user_query, k = 1)
    context = list(filter(lambda x: x[1] >= 0.6, context))

    if len(context) == 0:
        return "Eu não sou capaz de responder essa pergunta"
    
    return context

In [24]:
pergunta = "O que é baixo peso ao nascer?"

rag(pergunta)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_14196\3649881962.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'autor': 'Gabriel Miranda'}, page_content='é definido como peso inferior a 2500 gramas ao nascer independente do idade gestacional cite{WHO1. Quando comparados com bebês que nasceram acima dessa linha de corte, nascidos de baixo peso possuem maior risco de óbito, podem apresentar problemas de crescimento e desenvolvimento intelectual. E quando adultos, podem apresentar maior risco de obesidade, desenvolvimento de doenças do coração e diabetes cite{WHO1. Parto prematuro, má nutrição materna, consumo de alcool e cigarro durante a gestação são fatores já conhecidos que aumentam o risco de BPN. \\n\\nA partir do momento que o médico possui a informação de que determinada gravidez possui alto risco de gerar um bebê com baixo peso, uma série de medidas podem ser tomadas para evitar esse desfecho. Melhorar o status nutricional da mãe e o acesso a

'Eu não sou capaz de responder essa pergunta'